# Table of contents

* [Introduction](#Introduction)
    - [Motivation](#Motivation)
    - [Reconstruction versus Windows](#Reconstruction-versus-Windows)
    - [Reusable code](#Reusable-code)


* [DICOM-CT data 3D visualizations](#DICOM-CT-data-3D-visualizations)
    - [Reading data](#Reading-data)
    - [Windows and LUTs](#Windows-and-LUTs)
    - [3D reconstruction with manual windowing and MarchingCubes algorithm](#3D-reconstruction-with-manual-windowing-and-MarchingCubes-algorithm)

# Introduction

## Motivation
The main idea behind this notebook is 3D visualization of DICOM-CT data. The visualization of medical data of this type has great significance in enhancing the process of diagnosis. 

There are many popular libraries and programs like 3D Slicer, Myrian and so on, which provide the similar 3D Visualization and Volumetry as a premium SaS product. These softwares are highly expensive and proprietary. 

In this notebook you will find: DICOM-CT visualization and 3D Visualization of the CT Images using a Open Source Algorithm.

**Warning: This notebooks presents images containing blood visualizations. If you are afraid of the sight of blood, you read on your own responsibility.**

## Project Phases

### Phase - 1

    1. Visualize the DICOM Images
    2. Analyse the DICOM Images and Convert pixel values to HU Units
    3. 3D Reconstruction of Liver and the major blood vessels

### Phase - 2

    1. Differentiate Tumour and Healthy Tissues depending on HU Threshold Values
    2. Augment 3D Images of Tumour, Liver and blood vessels in a single 3D Model
    3. Conduct Volumetry on the entire model

## Reconstruction and Windowing
* source: https://www.youtube.com/watch?v=KZld-5W99cI

### Windowing
Windows are the different ways that we can visually display the digital information from any set of CT slices and our wide variety of windows common windows that are used at:
* lung windows
* soft tissue windows (includes liver and other internal organs) 
* bone windows 
* brain windows 
* blood windows 
and many others.

Example windows:
![](https://www.stepwards.com/wp-content/uploads/2019/12/Screen-Shot-2019-12-28-at-4.20.41-PM.png)

### Reconstruction 
***Reconstruction*** require the image processing of the raw data commonly by the technologist, but sometimes by the radiologist using specialized software programs the data can be processed in different ways to spending depending on what data we wish to obtain from it or what tissues we wish to display. So typical reconstructions might include a soft tissue, a bone and a lung reconstruction. We can also reconstruct in multiple planes as well as the coronal and sagittal planes. We can do some specialized planes along the organ for any it along the plane of any organ of interest. We can do 3d reconstructions such as surface  reconstruction; maximum intensity projection reconstructions. 
Reconstructions are done usually by the technologists or by some separate type of software on a different workstation by the radiologist to allow the raw CT data to be manipulated into different projections or different slices depending on what our clinical question is. Example of this procedure is shown in the diagram below:

![](https://d3i71xaburhd42.cloudfront.net/d4fb82cbcfda18ebe327e7003ec81ac777666299/3-Figure1-1.png)


This project is focused on 3D Reconstruction of Liver. Liver is been segmented from the CT Images and the segmented liver images are used to construct the 3D Mesh Object. It is worth noting that this notebook is mainly focused on reconstruction.

# DICOM-CT data 3D visualizations
Let's move on to the main topic of this notebook. 3D visualization of medical data is a very broad topic. We will start with reading the data:

Sources:
* https://gist.github.com/somada141/38d313a65581341f23fd
* https://pyscience.wordpress.com/2014/09/11/surface-extraction-creating-a-mesh-from-pixel-data-using-python-and-vtk/

## Reading data

In [ ]:
from IPython.display import Image
import imageio
import os
import shutil
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import gc
from vtk.util import numpy_support
import numpy
import vtk
import numpy as np
import pydicom
import scipy.ndimage
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from os import listdir
from plotly.offline import iplot
import plotly.figure_factory as FF

In [ ]:
## vtk reading dicom
datadir = "Datasets/CT_Sample/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(datadir)
reader.Update()

## Windows and LUTs
Choosing the right parameters for Windowing is a job for radiologists. Therefore - regardless of the 3D rendering algorithm, we will display our data as points in 3D space with standard windowing. The data displayed below use standard LUTs color palettes for Slicer:

Sources: 
* https://github.com/pletzer/dicom-data-reader/blob/master/python/readVTK.py
* https://radiopaedia.org/articles/windowing-ct
* https://www.slicer.org/wiki/Documentation/4.1/SlicerApplication/LookupTables/GenericAnatomyColors
* https://github.com/Slicer/Slicer/blob/47b845f91ef332e6008af4a166b294912de6d052/Modules/Loadable/VolumeRendering/Logic/vtkSlicerVolumeRenderingLogic.cxx#L458-L540
* https://dgobbi.github.io/vtk-dicom/doc/api/image_display.html

### Axial view:

In [ ]:
N =  18
default_width = 512
default_height = 512

def vtk_show(renderer, width = default_width, height = default_height, filename = ""):

    renderWindow = vtk.vtkRenderWindow()
    
    renderWindow.SetOffScreenRendering(1)
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(width, height)
    renderWindow.Render()
     
    windowToImageFilter = vtk.vtkWindowToImageFilter()
    windowToImageFilter.SetInput(renderWindow)
    windowToImageFilter.Update()
     
    writer = vtk. vtkPNGWriter()
    
    if filename == "":
        writer.SetWriteToMemory(1)
        writer.SetInputConnection(windowToImageFilter.GetOutputPort())
        writer.Write()    
        return bytes(memoryview(writer.GetResult()))
    else:
        writer.SetFileName(filename+".png")
        writer.SetInputConnection(windowToImageFilter.GetOutputPort())
        writer.Write()    
        return None


def vtk_render_gif(renderer, N, name, Roll = False, Azimuth = False, Elevation = False, Actor = None, RotateX = False, RotateY = False, RotateZ = False, Zoom = 0, Dolly = 0, standard = True, width = default_width, height = default_height):    
    if standard:
        renderer.ResetCamera()
        camera = renderer.MakeCamera()
        renderer.ResetCameraClippingRange()
        camera.SetPosition(0,0,0)
    os.makedirs(name,exist_ok=True)
    
    if Zoom != 0:
        renderer.GetActiveCamera().Zoom(Zoom)
        
    if Dolly != 0:
        renderer.GetActiveCamera().Dolly(Dolly)
        
    #tmpN = 1
    if N >0: # render gif
        for fi in range(N):
            if Roll:
                renderer.GetActiveCamera().Roll(360//N) 
            if Azimuth:
                renderer.GetActiveCamera().Azimuth(360//N) 
            if Elevation:
                renderer.GetActiveCamera().Elevation(360//N)
            if Actor is not None:
                if RotateX:
                    Actor.RotateX(360//N)
                if RotateY:
                    Actor.RotateY(360//N)
                if RotateZ:
                    Actor.RotateZ(360//N)                    
            vtk_show(renderer,filename = name + "/shot"+str(fi), width = width, height = height)
        # render gif and cleanup
        img_list = []
        for fi in range(N):
            img_list.append(mpimg.imread(name + '/shot' + str(fi) + '.png'))
        shutil.rmtree(name)
        imageio.mimsave(name + ".gif", img_list, duration=0.5)


In [ ]:
windowing = {}
windowing['lungs'] = [1500,-600,64,123,147]
windowing['mediastinum'] = [350,50,255,244,209]
windowing['bones'] = [300,400,177,122,101]
windowing['blood'] = [5,80,216,101,79]

imageData = reader.GetOutput()
volumeMapper = vtk.vtkSmartVolumeMapper()
volumeMapper.SetInputData(imageData)
volumeProperty = vtk.vtkVolumeProperty()
volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

for cur_windowing in windowing:
    cur_w = windowing[cur_windowing]
    opacity_function = vtk.vtkPiecewiseFunction()
    opacity_function.AddPoint(cur_w[1]-cur_w[0]/2,   0.0)
    opacity_function.AddPoint(cur_w[1],   1.0)
    opacity_function.AddPoint(cur_w[1]+cur_w[0]/2,   0.0)
    volumeProperty.SetScalarOpacity(opacity_function)

    color_function = vtk.vtkColorTransferFunction()
    color_function.SetColorSpaceToDiverging()
    color_function.AddRGBPoint(cur_w[1]-cur_w[0]/2,0,0,0)
    color_function.AddRGBPoint(cur_w[1],cur_w[2],cur_w[3],cur_w[4])
    color_function.AddRGBPoint(cur_w[1]+cur_w[0]/2, 0,0,0)

    volumeProperty.SetColor(color_function)

    volume = vtk.vtkVolume()
    volume.SetMapper(volumeMapper)
    volume.SetProperty(volumeProperty)

    renderer = vtk.vtkRenderer();
    volumeMapper.SetRequestedRenderModeToRayCast()
    renderer.AddViewProp(volume)
    
    renderer.ResetCamera()
    renderer.SetBackground(1,1,1);
    renderer.ResetCamera()
    renderer.ResetCameraClippingRange()
    camera = renderer.MakeCamera()

    camera.SetPosition(0,0,0)
    camera = renderer.GetActiveCamera()
    camera.Dolly(1.5)


    camera.Roll(360)
    name = cur_windowing + '_top'
    vtk_render_gif(renderer, N = 1 ,name =  name, standard = False)
    
    name = cur_windowing + '_front'
    camera.Elevation(240)
    camera.Elevation(20)
    vtk_render_gif(renderer, N = 1 ,name =  name, standard = False)
    
plt.rcParams["figure.figsize"] = (40,40)
idp = 0 
for cur_windowing in windowing:
    idp += 1
    plt.subplot(len(windowing),4, idp)
    try:
        im = mpimg.imread(cur_windowing+'_top.gif') 
        plt.imshow(im) 
        plt.title('Windowing: ' + cur_windowing, fontsize =20)
    except:
        pass

### Coronal view:

In [ ]:
plt.rcParams["figure.figsize"] = (40,40)
idp = 0 
for cur_windowing in windowing:
    idp += 1
    plt.subplot(len(windowing),4, idp)
    try:
        im = mpimg.imread(cur_windowing+'_front.gif') 
        plt.imshow(im) 
        plt.title('Windowing: ' + cur_windowing, fontsize =20)
    except:
        pass

## 3D reconstruction with manual windowing and MarchingCubes algorithm

In [ ]:
#File path for full upper body CT Image
CT_Upper = "Datasets/CT_Sample/"

In [ ]:
#File paths for segmented images of Patient-1
LiverPath = "Datasets/Patient_Two/Segmented_Liver/"
TumourPath = "Datasets/Patient_Two/Segmented_Tumour/"
ArteryPath = "Datasets/Patient_Two/Artery/"
VeinPath = "Datasets/Patient_Two/Hepatic_Venous/"

In [ ]:
#Get the first image from the dataset
sample_image = pydicom.dcmread(LiverPath + "\\" + listdir(LiverPath)[0]) 

In [ ]:
#CT Image Metadata
sample_image

In [ ]:
#Read all Dicom Images
def LoadDicomImages(Path):
    slices = [pydicom.dcmread(Path + "\\" + file) for file in listdir(Path)] #Get All Images
    #Sort the Images based on ImagePositionPatient attribute of the CT Images
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2])) 
    return slices

In [ ]:
#Calculate Hounsfields Values
def FindHUValues(slices):
    images = np.stack([file.pixel_array for file in slices])
    # Convert to int16 (from sometimes int16), 
	# should be possible as values should always be low enough (<32k)
    images = images.astype(np.int16)
    
    #Following is ignored - Should be used when the SliceThickness is less than 1
    # Set outside-of-scan pixels to 1
	# The intercept is usually -1024, so air is approximately 0
    images[images == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = slices[0].RescaleIntercept
    slope = slices[0].RescaleSlope
    if slope != 1:
        images = slope * images.astype(np.float64)
        images = images.astype(np.int16)
    images += np.int16(intercept)
        
    return np.array(images, dtype=np.int16)

A scan may have a pixel spacing of [2.5, 0.5, 0.5], which means that the distance between slices is 2.5 millimeters. For a different scan this may be [1.5, 0.725, 0.725], this can be problematic for automatic analysis and 3D Model Generation.

A common method of dealing with this is resampling the full dataset to a certain isotropic resolution. If we choose to resample everything to 1mm 1mm 1mm pixels (arbitrary) we can generate 3D model without worrying about learning zoom/slice thickness invariance.

Whilst this may seem like a very simple step, it has quite some edge cases due to rounding. Also, it takes quite a while.

In [ ]:
#Resampling driver for Stacking the images in the right positions
def ResampleImages(image, scan, new_spacing=[1,1,1], label = ""):
    print(label)
    print("Shape before re-sampling: ", image.shape)
    #Calculate the spacing between the CT Images from the Meta data
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)
    #Apply Resize Factor (new_spacing - Can be changed and try out different values and determine theoptimum by experimentation)
    resize_factor = spacing / new_spacing
    new_shape = np.round(image.shape * resize_factor)
    #Calculate new shape factor
    rounded_resize_factor = new_shape / image.shape
    rounded_new_spacing = spacing / rounded_resize_factor
    #Carry out Image Interpolation - in Nearest Mode
    #Mode: Points outside the boundaries of the input are filled according to the given mode (â€˜constantâ€™, â€˜nearestâ€™, â€˜reflectâ€™ or â€˜wrapâ€™)
    image = scipy.ndimage.interpolation.zoom(image, rounded_resize_factor, mode='nearest')
    print("Shape after re-sampling: ", image.shape)
    print("")
    return image, rounded_new_spacing

In [ ]:
#Get the corresponding Images
LiverImages = LoadDicomImages(LiverPath)
ArteryImages = LoadDicomImages(ArteryPath)
HepaticVenousImages = LoadDicomImages(VeinPath)
TumourImages = LoadDicomImages(TumourPath)
UpperLimbImages = LoadDicomImages(CT_Upper)

In [ ]:
#Get the HU Values of the images
LiverHU = FindHUValues(LiverImages)
ArteryHU = FindHUValues(ArteryImages)
HepaticVenousHU = FindHUValues(HepaticVenousImages)
TumourHU = FindHUValues(TumourImages)
UpperLimbHU = FindHUValues(UpperLimbImages)

In [ ]:
#Liver Image in HU Units
LiverHU[30][300]

HU Values Distribution across Upper Limb CT Image

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
plt.hist(UpperLimbHU.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

HU Values Distribution across Liver CT Image

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
plt.hist(LiverHU.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(LiverHU[50], cmap=plt.cm.gray)
plt.show()

In [ ]:
plt.hist(LiverHU.flatten(), bins=80, color='c', range=[1, 200])
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(LiverHU[50], cmap=plt.cm.gray)
plt.show()

In [ ]:
#Resample all the Images
ResampledLiver, LiverSpacing= ResampleImages(LiverHU, LiverImages, label = "Liver")
ResampledArtery, ArterySpacing= ResampleImages(ArteryHU, ArteryImages, label = "Artery")
ResampledHepaticVein, HepaticVeinSpacing= ResampleImages(HepaticVenousHU, HepaticVenousImages, label = "Hepatic Vein")
ResampledTumour, TumourSpaing = ResampleImages(TumourHU, TumourImages, label = "Tumour")
ResampledUpperLimb, UpperLimbSpacing= ResampleImages(UpperLimbHU, UpperLimbImages, label = "Upper Limb")

In [ ]:
#Rotate the 3D Mesh so that the horizontal stack will be converted into a vertical stack
Liver = ResampledLiver.transpose(2,1,0)
Artery = ResampledArtery.transpose(2,1,0)
HepaticVein = ResampledHepaticVein.transpose(2,1,0)
Tumour = ResampledTumour.transpose(2,1,0)
UpperLimb = ResampledUpperLimb.transpose(2,1,0)

To Visualize a single set of images

In [ ]:
#Convert the sampled Co-ordinates into 3D Mesh
def ViewMesh(sampledImages, threshold=30, title = "", plot_edges = False, 
             colormap=['rgba(255, 0, 0, 0.5)','rgba(255, 0, 0, 0.5)']):
    #Marching Cubes Lewiner
    #Change the threshold and other default values of the function to experiment different results
    #Returns Vertices, faces
    verts, faces,_,_ = measure.marching_cubes(sampledImages, threshold)
    #Convert vertices and 3D points array
    x,y,z = zip(*verts)
    #Create a 3D figure map
    figure = FF.create_trisurf(x=x, y=y, z=z, 
                            plot_edges=plot_edges,
                            colormap=colormap,
                            simplices=faces,
                            backgroundcolor='rgb(64, 64, 64)',
                            title=title)
    figure['data'][0].update(opacity=0.5)
    iplot(figure) 

Combine n number of images to create a 3D Model

In [ ]:
#Convert the sampled Co-ordinates into 3D Mesh
def ViewCombinedMesh(sampledImages, colorMap = [], threshold=30, title = "", plot_edges = False):
    #Marching Cubes Lewiner
    #Change the threshold and other default values of the function to experiment different results
    #Returns Vertices, faces
    figures = []    
    for i in range(len(sampledImages)):
        verts, faces,_,_ = measure.marching_cubes(sampledImages[i], threshold)
        #Convert vertices and 3D points array
        x,y,z = zip(*verts)
        #Create a 3D figure map
        if len(colorMap)>=i+1:
            currentMap = colorMap[i]
        else:
            currentMap = ['rgba(255, 0, 0, 0.5)','rgba(255, 0, 0, 0.5)']    
        figure = FF.create_trisurf(x=x, y=y, z=z, 
                            plot_edges=plot_edges,
                            colormap=currentMap,
                            simplices=faces,
                            backgroundcolor='rgb(64, 64, 64)',
                            title=(title if i==0 else ""))
        figure['data'][0].update(opacity=0.5)
        figures.append(figure.data[0])
        figures.append(figure.data[1])
    iplot(figures)
    

# 3D Visualization of Liver

In [ ]:
#Plot the 3D Model of the Liver
ViewMesh(Liver, threshold=35)

# 3D Visualization of Upper Limb

In [ ]:
#Plot the 3D Model of the Upper Limb
ViewMesh(UpperLimb, threshold=25, colormap=['rgba(255, 0, 255, 0.5)','rgba(255, 0, 255, 0.5)'])

# 3D Visualization of Liver and Tumour

In [ ]:
ViewCombinedMesh([Liver, Tumour], threshold = 30, title = "3D Model", colorMap = [
    ['rgba(255, 0, 0, 0.5)','rgba(255, 0, 0, 0.5)'],
    ['rgba(0, 0, 255, 0.5)','rgba(0, 0, 255, 0.5)']
])

# 3D visualization Liver(Full CT Input) 

In [ ]:
ViewCombinedMesh([Liver, Tumour, Artery, HepaticVein], threshold = 30, title = "3D Model", colorMap = [
    ['rgba(255, 0, 0, 0.5)','rgba(255, 0, 0, 0.5)'],
    ['rgba(0, 0, 255, 0.5)','rgba(0, 0, 255, 0.5)'],
    ['rgba(0, 255, 0, 0.5)','rgba(0, 255, 0, 0.5)'],
    ['rgba(255, 255, 0, 0.5)','rgba(255, 255, 0, 0.5)'],
])

In [ ]:
#Segregating Tumour into portions based on HU Value Threshold
MildHU = np.where(TumourHU <= 46, 0, TumourHU)

ModerateHUDummy = np.where(TumourHU <= 91, 0, TumourHU)
ModerateHU = np.where(ModerateHUDummy >= 46, 0, ModerateHUDummy)

AggressiveHU = np.where(TumourHU >= 91, 0, TumourHU)

#The above variations can be also used to create a the 3D Model. 
#This follows same procedure, from Resampling to 3D Model Generation.

# Volumetry

In [ ]:
def FindVolume(slices, hu_values):
    #slices - Image Slices
    #hu_values - Corresponding HU Values
    volume = 0
    for i in range(len(hu_values)):
        for j in range(len(hu_values[i])):
            #Calculate Surface area of every 2D Image from the pixel spacing and the corresponding HU Values
            surface_area = (slices[i].PixelSpacing[0] * slices[i].PixelSpacing[1] * np.count_nonzero(hu_values[i,j]))
            #Scale the area across SliceThickness
            volume += (slices[i].SliceThickness * surface_area)
    return (volume/1000)

In [ ]:
"Liver Volume: "+ str(FindVolume(LiverImages, LiverHU)) + " cc"

In [ ]:
"Tumour Volume: "+ str(FindVolume(TumourImages, TumourHU)) + " cc"

In [ ]:
"Healthy Liver Volume: "+ str(FindVolume(LiverImages, LiverHU)-FindVolume(TumourImages, TumourHU)) + " cc"

In [ ]:
"Artery Volume: "+ str(FindVolume(ArteryImages, ArteryHU)) + " cc"

In [ ]:
"HepaticVein Volume: "+ str(FindVolume(HepaticVenousImages, HepaticVenousHU)) + " cc"